In [1]:
import os, sys
import subprocess
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from pandas.tools.plotting import table
import seaborn as sns
from IPython.display import clear_output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
def Label_Test_Viewplane_A4C(number_of_set):
    clear_output()
    datafile_dir="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/Data.xls"
    #loading the name of patient files as per view
    Shuffled_A4C=pd.read_excel(datafile_dir, sheetname='Test_A4C', header=0)
    Shuffled_A2C=pd.read_excel(datafile_dir, sheetname='Test_A2C', header=0)
    Shuffled_PLA=pd.read_excel(datafile_dir, sheetname='Test_PLA', header=0)
            
    print "SAVING "+number_of_set
    print "SAVING A4C"
    for i in xrange(0,(Shuffled_A4C.size/4)):
        patnumber=Shuffled_A4C.iloc[i,0]
        Videonumber=Shuffled_A4C.iloc[i,1]
        Totalframes=Shuffled_A4C.iloc[i,2]
        Viewplane=Shuffled_A4C.iloc[i,3]
        score1=np.zeros((1,3))
        score2=np.zeros((1,3))
        score3=np.zeros((1,3))
        clear_output()
        for x in xrange(0,Totalframes):
            tf.reset_default_graph()
            k=x+1
            image_path="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/TEST/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            image_data = tf.gfile.FastGFile(image_path, 'rb').read()
            TFLABEL_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OL.txt"
            TFOG_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OG.pb"
            label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile(TFLABEL_DIR)]
            # Unpersists graph from file
            with tf.gfile.FastGFile(TFOG_DIR, 'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
                tf.import_graph_def(graph_def, name='')
            with tf.Session() as sess:
            # Feed the image_data as input to the graph and get first prediction
                    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
                    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
            predictions_max=(predictions == predictions.max(axis=1, keepdims=True)).astype(int)
            score1=np.add(score1,predictions).reshape(1,3)
            predictions_normalized = normalize(predictions, norm='max')
            score2=np.add(score2,predictions_normalized).reshape(1,3)
            score3=np.add(score3,predictions_max.astype(int)).reshape(1,3)
            perc1=np.multiply(np.divide(score1,np.sum(score1)),100).reshape(1,3)
            perc2=np.multiply(np.divide(score2,np.sum(score2)),100).reshape(1,3)
            perc3=np.multiply(np.divide(score3,np.sum(score3)),100).reshape(1,3)
            labels=label_lines
            label_lines=np.array(label_lines).reshape(1,3)
      
            print ('%s %s Sr.no %d/%d Patient number %d Video number %d Total_frames %d completed %d' %(number_of_set,Viewplane,i+1,(Shuffled_A4C.size/4),patnumber,Videonumber,Totalframes,k))
        Result=np.concatenate((label_lines,score1,perc1,score2,perc2,score3,perc3),axis=0)
        Result2=pd.DataFrame(data=Result[1:,0:],columns=Result[0,0:],index=["PS","PS%","PSM","PSM%","MS","MS%"])
        Result2=Result2.convert_objects(convert_numeric=True).round(2)
       
        print Result2
        savedir_append="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/RESULTS/"
        sizes = Result2.iloc[5,:]
        colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','red','green']
        explode = (0.1, 0, 0, 0)  # explode 1st slice
        Result2.plot.bar()
        barfname=str(savedir_append)+"BAR/BAR_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(barfname)
        plt.close()
        sns.heatmap(Result2,annot=True, fmt="2.2f")
        heatfname=str(savedir_append)+"HEAT/HEAT_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(heatfname)
        plt.close()

In [3]:
def Label_Test_Viewplane_A2C(number_of_set):
    clear_output()
    datafile_dir="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/Data.xls"
    #loading the name of patient files as per view
    Shuffled_A2C=pd.read_excel(datafile_dir, sheetname='Test_A2C', header=0)
                
    print "SAVING "+number_of_set
    print "SAVING A2C"
    for i in xrange(0,(Shuffled_A2C.size/4)):
        patnumber=Shuffled_A2C.iloc[i,0]
        Videonumber=Shuffled_A2C.iloc[i,1]
        Totalframes=Shuffled_A2C.iloc[i,2]
        Viewplane=Shuffled_A2C.iloc[i,3]
        score1=np.zeros((1,3))
        score2=np.zeros((1,3))
        score3=np.zeros((1,3))
        clear_output()
        for x in xrange(0,Totalframes):
            tf.reset_default_graph()
            k=x+1
            image_path="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/TEST/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            image_data = tf.gfile.FastGFile(image_path, 'rb').read()
            TFLABEL_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OL.txt"
            TFOG_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OG.pb"
            label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile(TFLABEL_DIR)]
            # Unpersists graph from file
            with tf.gfile.FastGFile(TFOG_DIR, 'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
                tf.import_graph_def(graph_def, name='')
            with tf.Session() as sess:
            # Feed the image_data as input to the graph and get first prediction
                    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
                    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
            predictions_max=(predictions == predictions.max(axis=1, keepdims=True)).astype(int)
            score1=np.add(score1,predictions).reshape(1,3)
            predictions_normalized = normalize(predictions, norm='max')
            score2=np.add(score2,predictions_normalized).reshape(1,3)
            score3=np.add(score3,predictions_max.astype(int)).reshape(1,3)
            perc1=np.multiply(np.divide(score1,np.sum(score1)),100).reshape(1,3)
            perc2=np.multiply(np.divide(score2,np.sum(score2)),100).reshape(1,3)
            perc3=np.multiply(np.divide(score3,np.sum(score3)),100).reshape(1,3)
            labels=label_lines
            label_lines=np.array(label_lines).reshape(1,3)
            print ('%s %s Sr.no %d/%d Patient number %d Video number %d Total_frames %d completed %d' %(number_of_set,Viewplane,i+1,(Shuffled_A2C.size/4),patnumber,Videonumber,Totalframes,k))
        Result=np.concatenate((label_lines,score1,perc1,score2,perc2,score3,perc3),axis=0)
        Result2=pd.DataFrame(data=Result[1:,0:],columns=Result[0,0:],index=["PS","PS%","PSM","PSM%","MS","MS%"])
        Result2=Result2.convert_objects(convert_numeric=True).round(2)
       
        print Result2
        savedir_append="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/RESULTS/"
        
        sizes = Result2.iloc[5,:]
        colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','red','green']
        explode = (0.1, 0, 0, 0)  # explode 1st slice
        Result2.plot.bar()
        barfname=str(savedir_append)+"BAR/BAR_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(barfname)
        plt.close()
        sns.heatmap(Result2,annot=True, fmt="2.2f")
        heatfname=str(savedir_append)+"HEAT/HEAT_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(heatfname)
        plt.close()

In [4]:
def Label_Test_Viewplane_PLA(number_of_set):
    clear_output()
    datafile_dir="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/Data.xls"
    #loading the name of patient files as per view
    Shuffled_PLA=pd.read_excel(datafile_dir, sheetname='Test_PLA', header=0)
            
    print "SAVING "+number_of_set
    print "SAVING PLA"
    for i in xrange(0,(Shuffled_PLA.size/4)):
        patnumber=Shuffled_PLA.iloc[i,0]
        Videonumber=Shuffled_PLA.iloc[i,1]
        Totalframes=Shuffled_PLA.iloc[i,2]
        Viewplane=Shuffled_PLA.iloc[i,3]
        score1=np.zeros((1,3))
        score2=np.zeros((1,3))
        score3=np.zeros((1,3))
        clear_output()
        for x in xrange(0,Totalframes):
            tf.reset_default_graph()
            k=x+1
            image_path="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/TEST/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            image_data = tf.gfile.FastGFile(image_path, 'rb').read()
            TFLABEL_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OL.txt"
            TFOG_DIR="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/"+str(number_of_set)+"_OG.pb"
            label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile(TFLABEL_DIR)]
            # Unpersists graph from file
            with tf.gfile.FastGFile(TFOG_DIR, 'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
                tf.import_graph_def(graph_def, name='')
            with tf.Session() as sess:
            # Feed the image_data as input to the graph and get first prediction
                    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
                    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
            predictions_max=(predictions == predictions.max(axis=1, keepdims=True)).astype(int)
            score1=np.add(score1,predictions).reshape(1,3)
            predictions_normalized = normalize(predictions, norm='max')
            score2=np.add(score2,predictions_normalized).reshape(1,3)
            score3=np.add(score3,predictions_max.astype(int)).reshape(1,3)
            perc1=np.multiply(np.divide(score1,np.sum(score1)),100).reshape(1,3)
            perc2=np.multiply(np.divide(score2,np.sum(score2)),100).reshape(1,3)
            perc3=np.multiply(np.divide(score3,np.sum(score3)),100).reshape(1,3)
            labels=label_lines
            label_lines=np.array(label_lines).reshape(1,3)
            print ('%s %s Sr.no %d/%d Patient number %d Video number %d Total_frames %d completed %d' %(number_of_set,Viewplane,i+1,(Shuffled_PLA.size/4),patnumber,Videonumber,Totalframes,k))
        Result=np.concatenate((label_lines,score1,perc1,score2,perc2,score3,perc3),axis=0)
        Result2=pd.DataFrame(data=Result[1:,0:],columns=Result[0,0:],index=["PS","PS%","PSM","PSM%","MS","MS%"])
        Result2=Result2.convert_objects(convert_numeric=True).round(2)
       
        print Result2

        savedir_append="/data/Gurpreet/RUNS/VC_8/"+str(number_of_set)+"/RESULTS/"
        sizes = Result2.iloc[5,:]
        colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','red','green']
        explode = (0.1, 0, 0, 0)  # explode 1st slice
        Result2.plot.bar()
        barfname=str(savedir_append)+"BAR/BAR_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(barfname)
        plt.close()
        sns.heatmap(Result2,annot=True, fmt="2.2f")
        heatfname=str(savedir_append)+"HEAT/HEAT_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        plt.savefig(heatfname)
        plt.close()

In [5]:
for i in xrange(1,10):
    #i=9
    x=i+1
    number_of_set="SET"+str(x)
    Label_Test_Viewplane_A4C(number_of_set)
    Label_Test_Viewplane_A2C(number_of_set)
    Label_Test_Viewplane_PLA(number_of_set)

SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 1
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 2
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 3
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 4
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 5
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 6
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 7
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 8
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 9
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 10
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 completed 11
SET10 PLA Sr.no 12/12 Patient number 31 Video number 67 Total_frames 55 co